### 21장 : 네트워크 분석

In [1]:
users = [
    {"id":0, "name":"Hero"},
    {"id":1, "name":"Dunn"},
    {"id":2, "name":"Sue"},
    {"id":3, "name":"Chi"},
    {"id":4, "name":"Thor"},
    {"id":5, "name":"Clive"},
    {"id":6, "name":"Hicks"},
    {"id":7, "name":"Devin"},
    {"id":8, "name":"Kate"},
    {"id":9, "name":"Klein"}
]
friendships = [(0,1), (0,2), (1,2), (1,3), (2,3), (3,4), (4,5), (5,6), (5,7), (6,8), (7,8), (8,9)]
users

[{'id': 0, 'name': 'Hero'},
 {'id': 1, 'name': 'Dunn'},
 {'id': 2, 'name': 'Sue'},
 {'id': 3, 'name': 'Chi'},
 {'id': 4, 'name': 'Thor'},
 {'id': 5, 'name': 'Clive'},
 {'id': 6, 'name': 'Hicks'},
 {'id': 7, 'name': 'Devin'},
 {'id': 8, 'name': 'Kate'},
 {'id': 9, 'name': 'Klein'}]

In [2]:
# 친구 입력하기
for user in users:
    user["friends"] = []

for i,j in friendships :
    users[i]["friends"].append(users[j]["id"])
    users[j]["friends"].append(users[i]["id"])
    
users

[{'friends': [1, 2], 'id': 0, 'name': 'Hero'},
 {'friends': [0, 2, 3], 'id': 1, 'name': 'Dunn'},
 {'friends': [0, 1, 3], 'id': 2, 'name': 'Sue'},
 {'friends': [1, 2, 4], 'id': 3, 'name': 'Chi'},
 {'friends': [3, 5], 'id': 4, 'name': 'Thor'},
 {'friends': [4, 6, 7], 'id': 5, 'name': 'Clive'},
 {'friends': [5, 8], 'id': 6, 'name': 'Hicks'},
 {'friends': [5, 8], 'id': 7, 'name': 'Devin'},
 {'friends': [6, 7, 9], 'id': 8, 'name': 'Kate'},
 {'friends': [8], 'id': 9, 'name': 'Klein'}]

In [3]:
# 친구가 가장 많은/적은 사람
number_of_friends = [(user["id"], user["name"], len(user["friends"])) for user in users]
sorted_number_of_friends = sorted(number_of_friends, key=lambda user: user[2], reverse=True)
sorted_number_of_friends

[(1, 'Dunn', 3),
 (2, 'Sue', 3),
 (3, 'Chi', 3),
 (5, 'Clive', 3),
 (8, 'Kate', 3),
 (0, 'Hero', 2),
 (4, 'Thor', 2),
 (6, 'Hicks', 2),
 (7, 'Devin', 2),
 (9, 'Klein', 1)]

- 많은 사람들과 연결되어있는 Dunn, Sue, Chi, Clive, Kate가 연결 중심성(degree centrality)가 높다. 
---
- 그러나 연결되어있는 네트워크를 보면 연결 수는 2이지만 양쪽을 연결하는 Thor가 주요 연결고리로 생각될 수 있다.
- ![21-1](https://github.com/jsj267/Data-Science-from-Scratch/blob/master/image/21_1.jpg?raw=true)
- 그래서 대안으로 사용할 수 있는 지수 중 하나가 매개 중심성(betweenness centrality)
- 매개 중심성이란 '임의의 두 사람 사이의 최단 경로상에서 얼마나 빈번하게 등장하는가'를 나타내는 지수.
- 즉, 노드 i의 매개 중심성은 다른 모든 임의의 노드 j,k에 대하여, (i를 지나는 경로 수) / (j,k쌍의 최단 경로 수)

- 매개 중심성을 계산하기 위해 BFS(breadth first search)를 사용하여 구하자
    + 교재에서는 ```users["friends"]```에 각 user를 통채로 넣어서 알고리즘을 구현했으나, 나는 ```users["friends"]```에 friend의 id만 넣어서 알고리즘을 구현했다.
        * 그 과정에서 각 user의 friend 데이터가 필요했는데, 각 user id로 friend의 id를 찾기 위한 ```friends_dict``` dictionary를 만들어서 사용했다. 

In [23]:
friends_dict = {}
for user in users:
    friends_dict.update({ user["id"] : user["friends"]})
friends_dict

{0: [1, 2],
 1: [0, 2, 3],
 2: [0, 1, 3],
 3: [1, 2, 4],
 4: [3, 5],
 5: [4, 6, 7],
 6: [5, 8],
 7: [5, 8],
 8: [6, 7, 9],
 9: [8]}

In [40]:
# 매개 중심성 계산하기 - 1) 최단경로 구하기 
from collections import deque

# 최단 경로를 값으로 가지고 있는 dict, shortest_paths_from 함수
def shortest_paths_from(from_user, friends_dict):
    # 초기화
    shortest_paths_to = { from_user["id"] : [[]] }
    # bfs를 위한 queue (이전 노드까지의 최단 경로를 이용하기 때문에 큐에 (이전노드, 현노드) 형태로 저장함)
    frontier = deque((from_user["id"], friend) for friend in from_user["friends"])

    # 경로를 다 탐색할 때까지
    while frontier:
        prev_user, user = frontier.popleft()
        user_id = user

        #재귀를 이용해서 이전 노드까지의 최단경로에 현 노드를 이어 경로 만들기
        paths_to_prev_user = shortest_paths_to[prev_user]
        new_paths_to_user = [path+[user_id] for path in paths_to_prev_user]

        #이미 최단경로 shortest_paths_to에 저장되어있는지 확인
        old_paths_to_user = shortest_paths_to.get(user_id, [])
            ##dict.get(key, default값) : dictionary에 key값이 있으면 그 값을 반환하고, 없으면 default값을 반환한다. default값을 적지 않으면 None 반환
        
        #이미 최단 경로가 있으면 그걸 일단 min_path로 쓰자.
        if old_paths_to_user:
            min_path_length = len(old_paths_to_user[0])
        else:
            min_path_length = float('inf')

        # '이전 노드까지의 최단경로 + 현 노드' 와 '이전에 구해놓은 최단경로' 를 비교해서 최종 최단 경로를 구한다.
        new_paths_to_user = [path 
                             for path in new_paths_to_user
                             if len(path) <= min_path_length
                             and path not in old_paths_to_user]

        shortest_paths_to[user_id] = old_paths_to_user + new_paths_to_user

        # 남은 이웃 노드를 queue에 추가. 
        frontier.extend((user, friend)
                       for friend in friends_dict[user]
                       if friend not in shortest_paths_to)
        
    return shortest_paths_to

In [41]:
# from_user = 0, 즉, id=0인 노드부터 최단거리 경로
shortest_paths_from(users[0], friends_dict)

{0: [[]],
 1: [[1]],
 2: [[2]],
 3: [[1, 3], [2, 3]],
 4: [[1, 3, 4], [2, 3, 4]],
 5: [[1, 3, 4, 5], [2, 3, 4, 5]],
 6: [[1, 3, 4, 5, 6], [2, 3, 4, 5, 6]],
 7: [[1, 3, 4, 5, 7], [2, 3, 4, 5, 7]],
 8: [[1, 3, 4, 5, 6, 8],
  [2, 3, 4, 5, 6, 8],
  [1, 3, 4, 5, 7, 8],
  [2, 3, 4, 5, 7, 8]],
 9: [[1, 3, 4, 5, 6, 8, 9],
  [2, 3, 4, 5, 6, 8, 9],
  [1, 3, 4, 5, 7, 8, 9],
  [2, 3, 4, 5, 7, 8, 9]]}

In [42]:
# 각 노드에 최단 거리를 저장, 매개 중심성 초기화
for user in users:
    user["shortest_paths"] = shortest_paths_from(user, friends_dict)
    user["betweenness_centrality"] = 0.0

In [43]:
# 매개중심성 계산(source가 시작점, target이 끝점)
for source in users:
    source_id = source["id"]
    for target_id, paths in source["shortest_paths"].items() : # dict.items() : key와 value를 묶어서 쌍으로 return해주는 함수
        if source_id < target_id : # 경로 한번만 세도록
            num_paths = len(paths)
            contrib = 1/num_paths
            for path in paths:
                for id in path:
                    if id not in [source_id, target_id]:
                        users[id]["betweenness_centrality"] += contrib

In [44]:
users

[{'betweenness_centrality': 0.0,
  'friends': [1, 2],
  'id': 0,
  'name': 'Hero',
  'shortest_paths': {0: [[]],
   1: [[1]],
   2: [[2]],
   3: [[1, 3], [2, 3]],
   4: [[1, 3, 4], [2, 3, 4]],
   5: [[1, 3, 4, 5], [2, 3, 4, 5]],
   6: [[1, 3, 4, 5, 6], [2, 3, 4, 5, 6]],
   7: [[1, 3, 4, 5, 7], [2, 3, 4, 5, 7]],
   8: [[1, 3, 4, 5, 6, 8],
    [2, 3, 4, 5, 6, 8],
    [1, 3, 4, 5, 7, 8],
    [2, 3, 4, 5, 7, 8]],
   9: [[1, 3, 4, 5, 6, 8, 9],
    [2, 3, 4, 5, 6, 8, 9],
    [1, 3, 4, 5, 7, 8, 9],
    [2, 3, 4, 5, 7, 8, 9]]}},
 {'betweenness_centrality': 3.5,
  'friends': [0, 2, 3],
  'id': 1,
  'name': 'Dunn',
  'shortest_paths': {0: [[0]],
   1: [[]],
   2: [[2]],
   3: [[3]],
   4: [[3, 4]],
   5: [[3, 4, 5]],
   6: [[3, 4, 5, 6]],
   7: [[3, 4, 5, 7]],
   8: [[3, 4, 5, 6, 8], [3, 4, 5, 7, 8]],
   9: [[3, 4, 5, 6, 8, 9], [3, 4, 5, 7, 8, 9]]}},
 {'betweenness_centrality': 3.5,
  'friends': [0, 1, 3],
  'id': 2,
  'name': 'Sue',
  'shortest_paths': {0: [[0]],
   1: [[1]],
   2: [[]],
   3: 

In [47]:
for user in users:
    print("id :", user["id"], "      betweenness_centrality :", user["betweenness_centrality"])

id : 0       betweenness_centrality : 0.0
id : 1       betweenness_centrality : 3.5
id : 2       betweenness_centrality : 3.5
id : 3       betweenness_centrality : 18.0
id : 4       betweenness_centrality : 20.0
id : 5       betweenness_centrality : 20.5
id : 6       betweenness_centrality : 6.0
id : 7       betweenness_centrality : 6.0
id : 8       betweenness_centrality : 8.5
id : 9       betweenness_centrality : 0.0


- ![21_2](https://github.com/jsj267/Data-Science-from-Scratch/blob/master/image/21_2.jpg?raw=true)
- 빈도가 높은 id=3,4,5 노드가 매개중심성이 높은 것을 볼 수 있다.
---

- 근접중심성(closeness centrality) : 1/(원접성farness)
- 원접성farness = 다른 모든 사용자의 최단 경로를 합한 값

In [48]:
def farness(user):
    return sum(len(paths[0]) for paths in user["shortest_paths"].values())

for user in users:
    user["closeness_centrality"] = 1/farness(user)
    print("id :", user["id"], "      closeness_centrality :", user["closeness_centrality"])

id : 0       closeness_centrality : 0.029411764705882353
id : 1       closeness_centrality : 0.037037037037037035
id : 2       closeness_centrality : 0.037037037037037035
id : 3       closeness_centrality : 0.045454545454545456
id : 4       closeness_centrality : 0.05
id : 5       closeness_centrality : 0.05
id : 6       closeness_centrality : 0.041666666666666664
id : 7       closeness_centrality : 0.041666666666666664
id : 8       closeness_centrality : 0.03571428571428571
id : 9       closeness_centrality : 0.027777777777777776


- ![21_3](https://github.com/jsj267/Data-Science-from-Scratch/blob/master/image/21_3.jpg?raw=true)
- 근접 중심성을 자주 사용하진 않고, 대신 더 쉽게 계산할 수 있는 고유벡터 중심성(eigenvector centrality)를 더 자주 사용한다고.

In [49]:
# 고유벡터 계산을 위한 행렬곱을 정의. 
## 행렬A의 i행, 행렬B의 j열
def matrix_product_entry(A, B, i, j) :
    return dot(get_row(A,i), get_column(B,j))

## 행렬 AB
def matrix_multiply(A, B):
    r1, c1 = shape(A)
    r2, c2 = shape(B)
    if c1!=r2 :
        raise ArithmetircError("incompatible shapes!")
    
    return make_matrix(r1, c2, partial(matrix_produc,t_entry, A, B))
    ##make_matrix function?